In [1]:
#Model
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import datetime
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
from keras import layers
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate, Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, AveragePooling2D, BatchNormalization, Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


#Server Deployment
from flask import Flask, send_file, request, jsonify, make_response, current_app
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from functools import update_wrapper, wraps
from pyngrok import ngrok
import requests

#Diagnosis
from ipywidgets import Button
from tkinter import Tk, filedialog
from IPython.display import clear_output, display
from beautifultable import BeautifulTable
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Environment variable
main_dir = 'D:/Data/HAM100000 - Harvard Dataset'
main_img_dir = main_dir + '/img_data'
preprocessed_data_dir = main_dir + '/preprocessed_data'
train_dir = preprocessed_data_dir + '/train'
test_dir = preprocessed_data_dir + '/val'
train_label = preprocessed_data_dir + '/train_label.csv'
val_label = preprocessed_data_dir + '/val_label.csv'

aug_label = preprocessed_data_dir + '/aug_label.csv'
aug_img_data_dir = main_dir + '/augmented_data'

# ModelPath
main_git_dir = 'D:/GithubCloneRepo/Skin-Disease-Detection-and-Segmentation-HAM100000'
experiment_dir = main_git_dir + '/Experiment'

dense_net_dir = experiment_dir + '/DenseNet201 - SoftAtt'
mobilenetV3_dir = experiment_dir + '/MobileNetV3'
model_densenet_path = dense_net_dir + '/densenetSoftAtt_10split_LRStr.hdf5'
model_mobilev3_path = mobilenetV3_dir + '/mobilenetv3LargeSoftAtt_10split_LRStr.hdf5'

In [4]:
train_df = pd.read_csv(train_label)
train_df.head()

,Unnamed: 0,image_id,dx,age,sex,localization
0,0,ISIC_0027419,bkl,80.0,male,scalp
1,1,ISIC_0025030,bkl,80.0,male,scalp
2,2,ISIC_0026769,bkl,80.0,male,scalp
3,3,ISIC_0025661,bkl,80.0,male,scalp
4,4,ISIC_0031633,bkl,75.0,male,ear


In [5]:
val_df = pd.read_csv(val_label)
val_df.head()

,Unnamed: 0,image_id,dx,age,sex,localization
0,2973,ISIC_0031531,bcc,70.0,male,back
1,483,ISIC_0026526,bkl,60.0,male,face
2,8870,ISIC_0026805,nv,75.0,female,lower extremity
3,1991,ISIC_0030756,mel,85.0,male,chest
4,8324,ISIC_0024834,nv,70.0,male,chest


In [6]:
age_input = tf.keras.Input(shape=(1,), name='age', dtype=tf.float32)
sex_input = tf.keras.Input(shape=(1,), name='sex', dtype=tf.string)
localizations_input = tf.keras.Input(shape=(1,), name='localization', dtype=tf.string)

inputs = {'age' : age_input,
         'sex' : sex_input,
         'local' : localizations_input}

In [7]:
norm = layers.Normalization()
norm.adapt(np.array(train_df[['age']]))
age_norm_input = norm(age_input)
age_norm_input

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization')>

In [8]:
# Sex input
sex_lookup = layers.StringLookup(vocabulary=np.unique(train_df['sex']))
sex_one_hot = layers.CategoryEncoding(num_tokens=sex_lookup.vocabulary_size())

preprocessed_sex_input = sex_lookup(sex_input)
preprocessed_sex_input = sex_one_hot(preprocessed_sex_input)
preprocessed_sex_input

<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'category_encoding')>

In [9]:
# Localization input
local_lookup = layers.StringLookup(vocabulary=np.unique(train_df['localization']))
local_one_hot = layers.CategoryEncoding(num_tokens=local_lookup.vocabulary_size())

preprocess_local_input = local_lookup(localizations_input)
preprocess_local_input = local_one_hot(preprocess_local_input)
preprocess_local_input

<KerasTensor: shape=(None, 16) dtype=float32 (created by layer 'category_encoding_1')>

In [10]:
preprocessed_inputs = [age_norm_input, preprocessed_sex_input, preprocess_local_input]
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

# preprocessed_Model = tf.keras.Model(inputs, preprocessed_inputs_cat)
# tf.keras.utils.plot_model(model = preprocessed_Model , rankdir="LR", dpi=72, show_shapes=True)

preprocessed_inputs_cat

<KerasTensor: shape=(None, 21) dtype=float32 (created by layer 'concatenate')>

In [11]:
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
import tensorflow as tf

In [12]:
class SoftAttention(Layer):
    def __init__(self,ch,m,concat_with_x=False,aggregate=False,**kwargs):
        self.channels=int(ch)
        self.multiheads = m
        self.aggregate_channels = aggregate
        self.concat_input_with_scaled = concat_with_x

        
        super(SoftAttention,self).__init__(**kwargs)

    def build(self,input_shape):

        self.i_shape = input_shape

        kernel_shape_conv3d = (self.channels, 3, 3) + (1, self.multiheads) # DHWC
    
        self.out_attention_maps_shape = input_shape[0:1]+(self.multiheads,)+input_shape[1:-1]
        
        if self.aggregate_channels==False:

            self.out_features_shape = input_shape[:-1]+(input_shape[-1]+(input_shape[-1]*self.multiheads),)
        else:
            if self.concat_input_with_scaled:
                self.out_features_shape = input_shape[:-1]+(input_shape[-1]*2,)
            else:
                self.out_features_shape = input_shape
        

        self.kernel_conv3d = self.add_weight(shape=kernel_shape_conv3d,
                                        initializer='he_uniform',
                                        name='kernel_conv3d')
        self.bias_conv3d = self.add_weight(shape=(self.multiheads,),
                                      initializer='zeros',
                                      name='bias_conv3d')

        super(SoftAttention, self).build(input_shape)

    def call(self, x):

        exp_x = K.expand_dims(x,axis=-1)

        c3d = K.conv3d(exp_x,
                     kernel=self.kernel_conv3d,
                     strides=(1,1,self.i_shape[-1]), padding='same', data_format='channels_last')
        conv3d = K.bias_add(c3d,
                        self.bias_conv3d)
        conv3d = kl.Activation('relu')(conv3d)

        conv3d = K.permute_dimensions(conv3d,pattern=(0,4,1,2,3))

        
        conv3d = K.squeeze(conv3d, axis=-1)
        conv3d = K.reshape(conv3d,shape=(-1, self.multiheads ,self.i_shape[1]*self.i_shape[2]))

        softmax_alpha = K.softmax(conv3d, axis=-1) 
        softmax_alpha = kl.Reshape(target_shape=(self.multiheads, self.i_shape[1],self.i_shape[2]))(softmax_alpha)

        
        if self.aggregate_channels==False:
            exp_softmax_alpha = K.expand_dims(softmax_alpha, axis=-1)       
            exp_softmax_alpha = K.permute_dimensions(exp_softmax_alpha,pattern=(0,2,3,1,4))
   
            x_exp = K.expand_dims(x,axis=-2)
   
            u = kl.Multiply()([exp_softmax_alpha, x_exp])   
  
            u = kl.Reshape(target_shape=(self.i_shape[1],self.i_shape[2],u.shape[-1]*u.shape[-2]))(u)

        else:
            exp_softmax_alpha = K.permute_dimensions(softmax_alpha,pattern=(0,2,3,1))

            exp_softmax_alpha = K.sum(exp_softmax_alpha,axis=-1)

            exp_softmax_alpha = K.expand_dims(exp_softmax_alpha, axis=-1)

            u = kl.Multiply()([exp_softmax_alpha, x])   

        if self.concat_input_with_scaled:
            o = kl.Concatenate(axis=-1)([u,x])
        else:
            o = u
        
        return [o, softmax_alpha]

    def compute_output_shape(self, input_shape): 
        return [self.out_features_shape, self.out_attention_maps_shape]

    
    def get_config(self):
        return super(SoftAttention,self).get_config()

In [13]:
densenet = tf.keras.applications.DenseNet201(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,

)

inputs['image'] = densenet.input

# Exclude the last 28 layers of the model.
densenet_output = densenet.layers[-28].output

In [14]:
attention_layer,map2 = SoftAttention(aggregate=True,m=16,concat_with_x=False,ch=int(densenet_output.shape[-1]),name='soft_attention')(densenet_output)
attention_layer=(MaxPooling2D(pool_size=(2, 2),padding="same")(attention_layer))
densenet_output=(MaxPooling2D(pool_size=(2, 2),padding="same")(densenet_output))

densenet_output = layers.Concatenate()([densenet_output, attention_layer])
densenet_output = Activation('relu')(densenet_output)
densenet_output = Dropout(0.5)(densenet_output)

In [15]:
output = Flatten()(densenet_output)
preprocessed_inputs_cat = (layers.Dense(4096, activation = 'relu')(preprocessed_inputs_cat))
output = layers.Concatenate()([output, preprocessed_inputs_cat])
output = Dense(7, activation='softmax')(output)
model = Model(inputs=inputs, outputs=output)

In [16]:
from tensorflow.keras import models
model.load_weights(model_densenet_path)

In [17]:
# test
img_path_test = train_dir + '/akiec/ISIC_0024329.jpg'
_img = cv2.imread(img_path_test)
sex_lst = ['male', 'female']
local_lst = ['back','lower extremity', 'trunk','upper extremity', 'abdomen','face','chest','foot','unknown','neck','scalp',                    
'hand', 'ear', 'genital', 'acral']  

_img = tf.keras.preprocessing.image.smart_resize(_img, (224,224))
_img = tf.keras.applications.mobilenet.preprocess_input(_img)

val_dict = {'age' : np.array([15]),
                'sex' : np.array([sex_lst[0]]),
                'local' : np.array([local_lst[0]]),
                'image' : np.array([_img])}

targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
targetnames_vn = ['Ung thư tế bào vảy khu trú', 'Ung thư biểu mô tế bào đấy', 'Dày sừng tiết bã', 'U sợi bì', 'Ung thư hắc tố da', 'Nốt ruồi', 'U mạch anh đào']

batch_size = 32 
with tf.device('/GPU:0'):
    predictions = model.predict(val_dict, steps=len(val_df)/batch_size, verbose=0)[0]
    print(predictions)
    print(type(predictions))
    
    ind = np.argpartition(predictions, 3)[-3:]
    print(ind)
    
    print(targetnames_vn[ind[2]])

[9.9836725e-01 4.4821843e-04 1.9043316e-04 4.1074329e-04 8.3446845e-05
 4.6250841e-04 3.7512174e-05]
<class 'numpy.ndarray'>
[1 5 0]
Ung thư tế bào vảy khu trú


In [18]:
# utils
def make_predict(_img_path, _sex, _age, _local):
    with tf.device('/GPU:0'):
        _img = cv2.imread(_img_path)
        _img = tf.keras.preprocessing.image.smart_resize(_img, (224,224))
        _img = tf.keras.applications.mobilenet.preprocess_input(_img)
        val_dict = {'age' : np.array([_age]),
                    'sex' : np.array([_sex]),
                    'local' : np.array([_local]),
                    'image' : np.array([_img])}
        predictions = model.predict(val_dict, verbose=0)[0]
        ind = np.argpartition(predictions, 3)[-3:]
        return {targetnames_vn[ind[2]] + " ({})".format(targetnames[ind[2]]) : round(predictions[ind[2]]*100, 2),
               targetnames_vn[ind[1]] + " ({})".format(targetnames[ind[1]]) : round(predictions[ind[1]]*100, 2),
               targetnames_vn[ind[0]] + " ({})".format(targetnames[ind[0]]) : round(predictions[ind[0]]*100, 2)}

def test(_img_path, _sex, _age, _local):
    test_dict = make_predict(_img_path, _sex, _age, _local)
    print(test_dict)
    
test(img_path_test, 'male', 15, 'back')

{'Ung thư tế bào vảy khu trú (akiec)': 99.84, 'Nốt ruồi (nv)': 0.05, 'Ung thư biểu mô tế bào đấy (bcc)': 0.04}


In [19]:
main_filename = 'D:/GithubCloneRepo/Skin-Disease-Detection-and-Segmentation-HAM100000/Server/IMG_Storage_File/{}.jpg'
test(main_filename.format('khoi'), 'male', 15, 'back')

{'Ung thư tế bào vảy khu trú (akiec)': 99.89, 'U sợi bì (df)': 0.1, 'Nốt ruồi (nv)': 0.0}


In [20]:
table = BeautifulTable()

def select_files(b):
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=True) # List of selected files will be set button's file attribute.
#     print(b.files[0]) # Print the list of files selected.
    test_dict = make_predict(b.files[0], 'male', 15, 'back')
    print("Diagnosis Result")
    img = mpimg.imread(b.files[0])
    imgplot = plt.imshow(img)
    table.rows.append(test_dict.keys())
    table.rows.append(list(test_dict.values()))
    print(table)
    
fileselect = Button(description="File select")
fileselect.on_click(select_files)

display(fileselect)

Button(description='File select', style=ButtonStyle())

In [ ]:
# Home page of server

app = Flask(__name__)
port_no = 5000

ngrok.set_auth_token("2AiAFtioJhVQYok9DwCvuPvi0z6_84sXtzLVkFzv1QH9sJVta")
public_url = ngrok.connect(port_no).public_url
print(f"To access the Global link please click {public_url}")

CORS(app)
# run_with_ngrok(app)

@app.route("/")
def home():
    return "<h1>SkinUNI Server</h1>"

@app.route('/upload/<path:filename>', methods= ['GET', 'POST'])
def upload_file(filename):
    if request.method == 'POST':
        f = request.files['files']
        filepath = main_filename.format(filename)
        f.save(filepath)
        return '200'
    else:
        return 'Upload ONLY'

@app.route('/diagnosis', methods= ['GET', 'POST'])
def diagnosis():
    if request.method == 'POST':
        json_data = request.json
        filepath = main_filename.format(json_data['path'])
        return jsonify(make_predict(filepath, json_data['sex'], json_data['age'], json_data['local']))
    
@app.route('/download/<path:filename>')
def get_file(filename):
    trans_filepath = main_filename.format(filename)
    return send_file(trans_filepath)
        
app.run(port=port_no)

To access the Global link please click http://42fa-2001-ee0-4001-b87f-48f8-9d43-965d-3ef9.ngrok.io
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jun/2022 23:15:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 23:15:13] "GET /download/test HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:00:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:00:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:00:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:00:32] "GET /?fbclid=IwAR2oqOTdT9lJzDXn9tgQTkYt4gX8qOEp7W0tnzMRejZIyBxwUAKZqt9gmCw HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:00:33] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Jun/2022 14:00:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:02:08] "GET /download/test HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:02:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Jun/2022 14:05:52] "POST /upload/img_1655535938150 HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:05:56] "OPTIONS /diagnosis HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:05:57] "POST /diagnosis HTTP/1.1" 2

[2022-06-18 14:17:24,896] ERROR in app: Exception on /diagnosis [POST]
Traceback (most recent call last):
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "d:\env\tf270\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\AI-INNOVATOR\AppData\Local\Temp\ipykernel_10268\3574682383.py", line 32, in diagnosis
    return jsonify(make_predict(filepath, json_data['sex'], json_data['age'], json_data[

127.0.0.1 - - [18/Jun/2022 14:19:30] "POST /diagnosis HTTP/1.1" 500 -
127.0.0.1 - - [18/Jun/2022 14:19:30] "OPTIONS /diagnosis HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:19:30] "OPTIONS /diagnosis HTTP/1.1" 200 -
[2022-06-18 14:19:30,613] ERROR in app: Exception on /diagnosis [POST]
Traceback (most recent call last):
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "d:\env\tf270\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.end

127.0.0.1 - - [18/Jun/2022 14:19:31] "POST /diagnosis HTTP/1.1" 500 -
127.0.0.1 - - [18/Jun/2022 14:19:32] "OPTIONS /diagnosis HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:19:32] "OPTIONS /diagnosis HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:19:32] "OPTIONS /diagnosis HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:19:32] "OPTIONS /diagnosis HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 14:19:32] "OPTIONS /diagnosis HTTP/1.1" 200 -
[2022-06-18 14:19:33,044] ERROR in app: Exception on /diagnosis [POST]
Traceback (most recent call last):
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "d:\env\tf270\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "d:\env\tf270\lib\site-packages\flask\

...

127.0.0.1 - - [18/Jun/2022 14:20:16] "POST /diagnosis HTTP/1.1" 500 -
[2022-06-18 14:20:16,875] ERROR in app: Exception on /diagnosis [POST]
Traceback (most recent call last):
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "d:\env\tf270\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\AI-INNOVATOR\AppData\Local\Temp\ipykernel_10268\3574682383.py", line 32, in diagnosis
    return jsonify

127.0.0.1 - - [18/Jun/2022 14:20:27] "POST /diagnosis HTTP/1.1" 500 -
[2022-06-18 14:20:27,270] ERROR in app: Exception on /diagnosis [POST]
Traceback (most recent call last):
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "d:\env\tf270\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\AI-INNOVATOR\AppData\Local\Temp\ipykernel_10268\3574682383.py", line 32, in diagnosis
    return jsonify

127.0.0.1 - - [18/Jun/2022 14:20:38] "POST /diagnosis HTTP/1.1" 500 -
[2022-06-18 14:20:38,973] ERROR in app: Exception on /diagnosis [POST]
Traceback (most recent call last):
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "d:\env\tf270\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\AI-INNOVATOR\AppData\Local\Temp\ipykernel_10268\3574682383.py", line 32, in diagnosis
    return jsonify

127.0.0.1 - - [18/Jun/2022 14:20:49] "POST /diagnosis HTTP/1.1" 500 -
127.0.0.1 - - [18/Jun/2022 14:20:49] "OPTIONS /diagnosis HTTP/1.1" 200 -
[2022-06-18 14:20:49,843] ERROR in app: Exception on /diagnosis [POST]
Traceback (most recent call last):
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "d:\env\tf270\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\AI-INNOVATOR\AppData\Local\Temp

127.0.0.1 - - [18/Jun/2022 14:20:50] "POST /diagnosis HTTP/1.1" 500 -
[2022-06-18 14:20:50,683] ERROR in app: Exception on /diagnosis [POST]
Traceback (most recent call last):
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "d:\env\tf270\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "d:\env\tf270\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\AI-INNOVATOR\AppData\Local\Temp\ipykernel_10268\3574682383.py", line 32, in diagnosis
    return jsonify